# Recreating figures from line game in Shafto 2014 

Setup: 

- 6 possible hypotheses $h_1$ through $h_6$ in hypothesis space 
- Uniform prior over hypotheses $P(h_{i}) = \frac{1}{6}$
- 12 data $d_1$ through $d_{12}$, where teacher reveals two segments that are labeled to be either inside the hypothesis or outside the hypothesis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

#sns.set()
sns.set_style('white')
sns.set_context('talk')

In [ ]:
# create hypotheses array
h = np.array([
    [1, 0, 0], 
    [1, 1, 0], 
    [1, 1, 1], 
    [0, 1, 0], 
    [0, 1, 1], 
    [0, 0, 1]
])

# create data array
d = np.array([
    [1, 1, np.nan],
    [1, 0, np.nan], 
    [0, 1, np.nan], 
    [0, 0, np.nan], 
    [1, np.nan, 1], 
    [1, np.nan, 0], 
    [0, np.nan, 1],
    [0, np.nan, 0],
    [np.nan, 1, 1],
    [np.nan, 1, 0], 
    [np.nan, 0, 1],
    [np.nan, 0, 0]
])

# create mask for hypothesis array for NaN values
d_mask = np.ma.masked_invalid(d)

## Finding probabilities for iteration 0: $P(d|h)$, random sampling

In [ ]:
d_possible = {}  # Set up dict of possible d for each h

rows = ['d_1', 'd_2', 'd_3', 'd_4', 
        'd_5', 'd_6', 'd_7', 'd_8', 
        'd_9', 'd_10', 'd_11', 'd_12']
columns = ['h_1', 'h_2', 'h_3', 'h_4', 'h_5', 'h_6']
df_0 = pd.DataFrame(index=rows, columns=columns).fillna(0)

# Loop over all combinations of h and d and fill dataframe with possible values
for row_h in range(h.shape[0]): 
    for row_d in range(d.shape[0]): 
        if np.array_equal(d[row_d][~d_mask.mask[row_d]], h[row_h][~d_mask.mask[row_d]]):
            d_possible.setdefault(row_h+1, []).append(row_d+1)
            df_0.iloc[row_d, row_h] = 1  # uniform distribution of data given each hypothesis
            
# Turn values into probabilities; each column sums up to 1
df_0 = df_0.div(df_0.sum(axis=0), axis=1)

In [ ]:
# View combinations: Keys are indices for d and values are indices for h
d_possible

In [ ]:
# Iteration 0 probabilities
df_0

In [ ]:
plt.figure(figsize=(4.8,7))
sns.heatmap(df_0, annot=True, linewidths=0.25)
plt.title('Iteration 0: $P_{teacher} (d|h)$')

plt.show()

## Probabilities for iteration 1

In [ ]:
df_1 = pd.DataFrame(index=rows, columns=columns).fillna(0)

Idea: 

$$ P_{learner}(h|d) = \frac{P_{teacher}(d|h) P(h)}{\sum_{h'} {P_{teacher}(d|h') P(h')}} $$

(prior probabilities of hypothesis are all $\frac{1}{6}$, so it cancels out)

$$ P_{teacher}(d|h) = \frac{P_{learner}(h|d) P(d)}{\sum_{d'} {P_{learner}(h|d')P(d')}} $$

(priors of d are the same for each iteration, too? should be one over number of possible data corresponding to each hypothesis) 

In [ ]:
# P(h|d) for learner 
df_1 = df_0.div(df_0.sum(axis=1), axis=0)
df_1

In [ ]:
# New P(d|h) for teacher
df_1 = df_1.div(df_1.sum(axis=0), axis=1)
df_1

In [ ]:
plt.figure(figsize=(5.5,8))
sns.heatmap(df_1, annot=True, linewidths=0.25)
plt.title('Iteration 1: $P_{teacher} (d|h)$')

plt.show()

## Iterations 2 and beyond

In [ ]:
def find_teacher_probabilities(n, df_0):
    '''
    given number of iterations n and P(d|h) matrix for iteration 0, find P(d|h) matrix after iteration n 
    '''
    n_iter = n
    df = df_0

    for n in range(n_iter): 
        df = df.div(df.sum(axis=1), axis=0)  # P(h|d)
        df = df.div(df.sum(axis=0), axis=1)  # P(d|h)
    
    return df

In [ ]:
df_2 = find_teacher_probabilities(2, df_0)

plt.figure(figsize=(5.5,8))
sns.heatmap(df_2, annot=True, linewidths=0.25)
plt.title('Iteration 2: $P_{teacher} (d|h)$')

plt.show()

Sanity check: $d_6$, iteration 2
$$ P_{teacher} (d_6 | h_2) = \frac{.33/.53}{(.33/.58) + (.33/.53) + (.33/.58)} \approx .354 $$

In [ ]:
# Iteration 500

df_500 = find_teacher_probabilities(500, df_0)

plt.figure(figsize=(5.5,8))
sns.heatmap(df_500, annot=True, linewidths=0.25)
plt.title('Iteration 500: $P_{teacher} (d|h)$')

plt.show()

## Plotting a few distributions over iterations

In [ ]:
probs_dict = {
    'd6h1': [df_0.loc['d_6', 'h_1']], 
    'd2h1': [df_0.loc['d_2', 'h_1']], 
    'd1h2': [df_0.loc['d_1', 'h_2']], 
    'd6h2': [df_0.loc['d_6', 'h_2']],
    'd1h3': [df_0.loc['d_1', 'h_3']],
    'd5h3': [df_0.loc['d_5', 'h_3']]
             }

for n in range(1,250):
    df = find_teacher_probabilities(n, df_0)
    probs_dict['d6h1'].append(df.loc['d_6', 'h_1'])
    probs_dict['d2h1'].append(df.loc['d_2', 'h_1'])
    probs_dict['d1h2'].append(df.loc['d_1', 'h_2'])
    probs_dict['d6h2'].append(df.loc['d_6', 'h_2'])
    probs_dict['d1h3'].append(df.loc['d_1', 'h_3'])
    probs_dict['d5h3'].append(df.loc['d_5', 'h_3'])

In [ ]:
df_probs = pd.DataFrame(probs_dict)
df_probs

In [ ]:
df_probs.iloc[:50, :].plot(xlabel='Iteration', ylabel='$P(d|h)$')
plt.title('Example probabilities over first 50 iterations')
plt.show()

## Entropy

Put all model predictions together into one matrix:

In [ ]:
mtx_0 = df_0.to_numpy()
mtx_1 = df_1.to_numpy()
mtx_500 = df_500.to_numpy()

posterior_mtx = np.stack([mtx_0, mtx_1, mtx_500],axis=-1)
print(posterior_mtx.shape)

Calculate $H(d|h)$:

$$
H(X) = -\sum_{i=1}^{n}\mathrm{P}(x_i)\mathrm{log}_b\mathrm{P}(x_i)
$$

In [ ]:
# Compute entropy
entropy_mtx = stats.entropy(posterior_mtx)

# Assemble into a tidy datframe
entropy_df = pd.DataFrame(entropy_mtx, 
                          index=['h_%i' % (i+1) for i in range(6)],
                          columns=['0', '1', '500'])

entropy_df.index.name = 'hypothesis' # Hypothesis column
entropy_df = entropy_df.reset_index()

entropy_df = pd.melt(entropy_df, id_vars=['hypothesis'], value_vars=['0','1','500'],
                     var_name='iterations', value_name='entropy')

entropy_df

In [ ]:
ax = sns.relplot(data=entropy_df, x='iterations', y='entropy', col='hypothesis', kind='line')
ax.set(xlabel='Iterations', ylabel='Entropy: $H(d|h)$')